# C++ Metaprogramming Snippets
This notebook demonstrates C++ metaprogramming techniques and useful code snippets using the xeus-cling kernel.

**Note:** To run C++ code in this notebook, select the xeus-cling C++ kernel (e.g., C++17) from the kernel menu.

## Compile-time Constants with `constexpr`
Use `constexpr` to define constants and functions evaluated at compile time.

In [ ]:
// Compile-time factorial using constexpr
#include <iostream>
constexpr int factorial(int n) {
    return n <= 1 ? 1 : (n * factorial(n - 1));
}

int main() {
    constexpr int val = factorial(5);
    std::cout << "Factorial of 5 at compile time: " << val << std::endl;
    return 0;
}

## Template Metaprogramming
Use templates to compute values at compile time and enable generic programming.

In [ ]:
// Template metaprogramming: factorial
#include <iostream>

template <int N>
struct Factorial {
    static constexpr int value = N * Factorial<N - 1>::value;
};

template <>
struct Factorial<0> {
    static constexpr int value = 1;
};

int main() {
    std::cout << "Factorial<5>::value = " << Factorial<5>::value << std::endl;
    return 0;
}

## SFINAE and Type Traits
Use SFINAE (Substitution Failure Is Not An Error) and type traits to enable or disable functions based on type properties.

In [ ]:
// SFINAE and type traits example
#include <iostream>
#include <type_traits>

template <typename T>
typename std::enable_if<std::is_integral<T>::value, void>::type
print_type(T value) {
    std::cout << value << " is integral" << std::endl;
}

template <typename T>
typename std::enable_if<!std::is_integral<T>::value, void>::type
print_type(T value) {
    std::cout << value << " is NOT integral" << std::endl;
}

int main() {
    print_type(42);      // integral
    print_type(3.14);    // not integral
    return 0;
}

## Variadic Templates and Fold Expressions
Use variadic templates and fold expressions to write functions that accept any number of arguments.

In [ ]:
// Variadic templates and fold expressions: sum of arguments
#include <iostream>

template <typename... Args>
auto sum(Args... args) {
    return (args + ...);
}

int main() {
    std::cout << "sum(1, 2, 3, 4) = " << sum(1, 2, 3, 4) << std::endl;
    std::cout << "sum(10, 20) = " << sum(10, 20) << std::endl;
    return 0;
}

In [ ]:
// Compile-time index of string in a list using constexpr and template recursion
#include <array>
#include <string_view>
#include <cstddef>

// Helper to compare string_views at compile time
constexpr bool sv_equal(std::string_view a, std::string_view b) {
    return a == b;
}

// Recursive constexpr function to find index
template <std::size_t N>
constexpr std::size_t index_of(std::string_view value, const std::array<std::string_view, N>& arr, std::size_t idx = 0) {
    return idx == N ? N : (sv_equal(arr[idx], value) ? idx : index_of(value, arr, idx + 1));
}

// Usage example
constexpr std::array<std::string_view, 4> fruits = {"apple", "banana", "cherry", "date"};
constexpr std::size_t idx = index_of("cherry", fruits);

static_assert(idx == 2, "Index of 'cherry' should be 2");

#include <iostream>
int main() {
    std::cout << "Index of 'cherry' is: " << idx << std::endl;
    return 0;
}

In [ ]:
is there a way to split a string at compile time in C++? the string is delimited by a '.'

Yes, you can split a string at compile time in C++ using template metaprogramming and constexpr functions. Below is an example of how to achieve this by defining a constexpr function that splits a string based on a delimiter (in this case, a dot '.').
```cpp
#include <array>
#include <string_view>     

constexpr std::size_t count_delimiters(std::string_view str, char delimiter) {
    std::size_t count = 0;
    for (char c : str) {
        if (c == delimiter) {
            ++count;
        }
    }
    return count;
}

constexpr std::array<std::string_view, 10> split_string(std::string_view str, char delimiter) {
    std::array<std::string_view, 10> result{};
    std::size_t start = 0;
    std::size_t index = 0;

    for (std::size_t i = 0; i < str.size(); ++i) {
        if (str[i] == delimiter) {
            result[index++] = str.substr(start, i - start);
            start = i + 1;
        }
    }
    result[index] = str.substr(start); // Add the last segment
    return result;
}

#include <iostream>

int main() {
    constexpr std::string_view input = "apple.banana.cherry.date";
    constexpr auto parts = split_string(input, '.');

    for (const auto& part : parts) {
        if (!part.empty()) { // Skip empty strings
            std::cout << part << std::endl;
        }
    }

    return 0;
}
``` 
This code defines a `split_string` function that takes a `std::string_view` and a delimiter character. It splits the string into parts and stores them in a `std::array`.The `count_delimiters` function counts how many times the delimiter appears in the string, which can be useful for determining the size of the resulting array. The `split_string` function iterates